In [1]:
import uproot
import awkward
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.metrics import roc_curve, auc, roc_auc_score
import torch
from torch.utils.data.dataloader import DataLoader
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F
import timeit
import os

# import tensorflow as tfcd
# import keras
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from sklearn.metrics import auc
# import seaborn as sns


In [2]:
workpath = os.getcwd()
file_train=uproot.open(workpath+'/'+'data/train_D02kpipi0vxVc-cont0p5.root')
file_test=uproot.open(workpath+'/'+'data/test_D02kpipi0vxVc-cont0p5.root')
tree_train=file_train['d0tree']
tree_test=file_test['d0tree']
df_train=tree_train.arrays(library="pd")
df_test=tree_test.arrays(library="pd")

In [3]:

torch.cuda.empty_cache()

In [4]:
df_train=df_train.drop(['vM','vpCMS','__index__'],axis=1)
df_test=df_test.drop(['vM','vpCMS','__index__'],axis=1)

In [5]:
X_train=df_train.drop(['isSignal'],axis=1)
Y_train=df_train['isSignal']
X_test  =df_test.drop(['isSignal'],axis=1)
Y_test=df_test['isSignal']

In [6]:
X_train=torch.tensor(X_train.values.astype(np.float32))
X_test=torch.tensor(X_test.values.astype(np.float32))
Y_train=torch.tensor(Y_train.values.astype(np.int64))
Y_test=torch.tensor(Y_test.values.astype(np.int64))
     


In [8]:
print('xtrain device name',X_train.device)
X_train.shape

xtrain device name cpu


torch.Size([40294, 12])

In [9]:
def device_available():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def feed_device(data,device):
    if isinstance(data, (list,tuple)):
        return [feed_device(x, device) for x in data]
    return data.to(device,non_blocking = True)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield feed_device(b, self.device)

    #def __len__(self):
      #  """Number of batches"""
     #   return len(self.dl)

In [10]:
class ANN_Model(nn.Module):
    def __init__(self,input_features=12,hidden1=60,hidden2=25,hidden3=30,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.f_connected3=nn.Linear(hidden2,hidden3)
        self.out=nn.Linear(hidden3,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=F.relu(self.f_connected3(x))
        #x=F.softmax(self.out(x))
        x=torch.sigmoid(self.out(x))
        return x

In [11]:
batch_size_train=40294
batch_size_test=37150
device = device_available()
device

device(type='cpu')

In [12]:
print('memory')
print(torch.cuda.memory_allocated())

memory
0


In [13]:
X_train.shape

torch.Size([40294, 12])

In [14]:
train_tensor = torch.utils.data.TensorDataset(X_train,Y_train)
test_tensor = torch.utils.data.TensorDataset(X_test,Y_test)

In [15]:
X_train_bt = DataLoader(train_tensor, batch_size_train,shuffle =True, num_workers = 3, pin_memory = True )
X_test_bt = DataLoader(test_tensor, batch_size_test,shuffle =True )#, num_workers = 4, pin_memory = True)
y_train_bt = DataLoader(Y_train, batch_size_train,shuffle =True , num_workers = 4, pin_memory = True)
y_test_bt = DataLoader(Y_test, batch_size_test,shuffle =True , num_workers = 4, pin_memory = True)

In [16]:
train_loader = DeviceDataLoader(X_train_bt, device)
y_train_bt = DeviceDataLoader(y_train_bt, device)
test_loader= DeviceDataLoader(X_test_bt, device)
y_test_bt = DeviceDataLoader(y_test_bt, device)


In [17]:
for i in train_loader:
    print(i[0].is_cuda)
    break

False


In [18]:
model=ANN_Model()
model=model.cuda()
for i in model.parameters():
    print(i.is_cuda)

AssertionError: Torch not compiled with CUDA enabled

In [19]:
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [20]:
X_train=X_train.values
X_test=X_test.values
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

TypeError: float() argument must be a string or a number, not 'builtin_function_or_method'

In [21]:
       
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))
def training_step( batch):
        images, labels = batch 
        out = model(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
def validation_step(batch):
        images, labels = batch 
        out =  model(images)                     # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
def validation_epoch_end(outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    


In [22]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [23]:
epochs=10

In [24]:
import time
#def evaluate(model, val_loader):
#    model.eval()
#    outputs = [validation_step(batch) for batch in val_loader]
#    return validation_epoch_end(outputs)
t1=time.time()
final=[]
epoch=20
result={}
torch.cuda.empty_cache()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
loss_function= nn.CrossEntropyLoss()

for i in range(epoch):
    optimizer.zero_grad()
    train_losses = []
    for inpt, target in tqdm(train_loader): 
        out = model.forward(inpt)
        
        #print(target)
        #print('out.shape:', out.shape)
        #print('out[0]:', out[0])
        loss = loss_function(out,target)
      
        #train_losses.append(loss)
       
        loss.backward()
        optimizer.step()
    print("Epoch number: {}  and the loss : {}".format(i,loss.item()))    
    #acc = accuracy(out, target) 
    #result = evaluate(model, train_loader)
    #result['train_loss']=torch.stack(train_losses).mean().item()
    #print("Epoch [{}], train_loss: {:.4f}".format(
    #        i, result['train_loss']))
    #final.append(result)
#(final)
t2=time.time()

0it [00:00, ?it/s]

Epoch number: 0  and the loss : 0.695013165473938


0it [00:00, ?it/s]

Epoch number: 1  and the loss : 0.6944658160209656


0it [00:00, ?it/s]

...

Epoch number: 17  and the loss : 0.6846414804458618


0it [00:00, ?it/s]

Epoch number: 18  and the loss : 0.6835363507270813


0it [00:00, ?it/s]

Epoch number: 19  and the loss : 0.6823260188102722


In [25]:
t2-t1

18.715291261672974

In [26]:
X_train=X_train.cuda()

AttributeError: 'builtin_function_or_method' object has no attribute 'cuda'

In [27]:
Y_train=Y_train.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [73]:
test =df_train[df_train['isSignal']==1]
test_x=test.drop(['isSignal'],axis=1)
test_y=test['isSignal']
test_x=torch.tensor(test_x.values.astype(np.float32))
test_y=torch.tensor(test_y.values.astype(np.int64))

In [74]:
test_x=test_x.cuda()
test_y=test_y.cuda()

In [75]:
out=model(test_x)
_, preds = torch.max(out, dim=1)
print("accuracy:",torch.tensor(torch.sum(preds == test_y).item() / len(preds)))


accuracy: tensor(0.8695)


In [76]:
t2-t1

19.216973304748535

In [77]:
t2-t1

19.216973304748535

In [78]:
X_test=X_test.cuda()
Y_test=y_test.cuda()
out=model(X_test)
_, preds = torch.max(out, dim=1)
print("accuracy:",torch.tensor(torch.sum(preds == Y_test).item() / len(preds)))

AttributeError: 'builtin_function_or_method' object has no attribute 'cuda'

In [79]:
y_test=y_test.astype(int)
correct=0
accuracy=accuracy_score(y_test,predictions)
print(accuracy)

NameError: name 'y_test' is not defined

In [35]:
preds = torch.max(out, dim=1)
print("accuracy:",torch.tensor(torch.sum(preds == y_test).item() / len(preds)))

NameError: name 'y_test' is not defined

In [36]:
with torch.no_grad():
    x= feed_device(X_test,device)
    y=model(x)
    print(y)
    preds  = torch.argmax(y,axis=1)
    print(preds)

AttributeError: 'builtin_function_or_method' object has no attribute 'to'

In [37]:
class load():
    def load_train_tensor(self,X_train,y_train):
        X_train=torch.tensor(X_train.values.astype(np.float32))     
        y_train=torch.tensor(y_train.values.astype(np.int64))
        train_tensor = torch.utils.data.TensorDataset(X_train,y_train)
        train_loader = DataLoader(train_tensor, 32,shuffle =True )
        return train_loader
    def load_test_tensor(self):
        X_test=torch.tensor(X_test.values.astype(np.float32))
        y_test=torch.tensor(y_test.values.astype(np.int64)) 
        test_tensor = torch.utils.data.TensorDataset(X_test,y_test)
        return test_tensor
    def load_train_batch(batch_size=32):
        train_loader = DataLoader(train_tensor, batch_size_train,shuffle =True )


In [38]:
class preprocess(nn.Module):   
    def training_step(self, batch):
        inp, labels = batch 
        out = model.forward(inp)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
          
    def test_step(self,batch):
        inp, labels = batch 
        out =  model(inp)                     # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = metric.accuracy(out, labels)           # Calculate accuracy
        return {'test_loss': loss.detach(), 'test_acc': acc}
    def test_epoch_end(self,outputs):
        batch_losses = [x['test_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['test_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'test_loss': epoch_loss.item(), 'test_acc': epoch_acc.item()}
    def epoch_end(self, epoch, result):
        print("Epoch [{}], test_loss: {:.4f}, test_acc: {:.4f}".format(epoch, result['test_loss'], result['test_acc']))
    

In [39]:
class ANN_Model(preprocess,load):
    def __init__(self,input_features=12,hidden1=60,hidden2=25,hidden3=30,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.f_connected3=nn.Linear(hidden2,hidden3)
        self.out=nn.Linear(hidden3,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=F.relu(self.f_connected3(x))
        #x=F.softmax(self.out(x))
        x=torch.sigmoid(self.out(x))
        return x

In [40]:
class metric():
    @torch.no_grad()
    def evaluate(model, test_loader):
        outputs = [model.test_step(batch) for batch in test_loader]
        return model.test_epoch_end(outputs)


    def fit(epochs, lr, model, train_loader, test_loader, opt_func=torch.optim.SGD):
        optimizer = opt_func(model.parameters(), lr)
        history = [] # for recording epoch-wise results

        for epoch in range(epochs):

            # Training Phase 
            for batch in train_loader:
                loss = model.training_step(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            # Validation phase
            result = metric.evaluate(model, test_loader)
            model.epoch_end(epoch, result)
            history.append(result)

        return history
    def accuracy(outputs, labels):
        _, preds = torch.max(outputs, dim=1)
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))


In [41]:
device=device_available()
device

device(type='cuda')

In [42]:
model=ANN_Model()
model=model.cuda()
for i in model.parameters():
    print(i.is_cuda)

True
True
True
True
True
True
True
True


In [43]:
x_train_bt=model.load_train_tensor(X_train,y_train)


NameError: name 'y_train' is not defined